In [2]:
# Importaciones de la biblioteca estándar
import sys
import time
import pickle
import sqlite3

# Importaciones de terceros
import numpy as np
import pandas as pd
import torch
from scipy import stats
import torchaudio
from scipy.signal import butter, sosfilt
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# Importaciones de módulos específicos del proyecto
from maad import sound, features

# Añadir el directorio utils al PATH para importar otros módulos si es necesario
sys.path.append("../utils/")
import indices as indices

# Configuración de la conexión a la base de datos
conn = sqlite3.connect('../results/results.db')
cursor = conn.cursor()


In [3]:
def butter_bandstop_filter(audio_signal, frequencies_start, frequencies_end, fs, order=9):
    """
    Apply a series of higher-order Butterworth bandstop filters to an audio signal.

    Parameters:
    - audio_signal: Numpy array containing the audio signal to be filtered.
    - frequencies_start: List of start frequencies for the bandstop filters.
    - frequencies_end: List of end frequencies for the bandstop filters.
    - fs: Sampling frequency of the audio signal.
    - order: Order of the Butterworth filters.

    Returns:
    - audio_signal: The filtered audio signal.
    """
    # Ensure the frequency lists are of the same length
    if len(frequencies_start) != len(frequencies_end):
        raise ValueError("Start and end frequency lists must have the same length.")

    # Design and apply each bandstop filter
    for start_freq, end_freq in zip(frequencies_start, frequencies_end):
        # Calculate the Nyquist frequency
        nyq = fs / 2.0
        # Calculate low and high cutoff frequencies for the bandstop filter
        low = start_freq / nyq
        if low == 0:
            low = 0.0001
        high = end_freq / nyq
        if high == 1:
            high = 0.9999
        # Design the Butterworth bandstop filter
        sos = butter(order, [low, high], btype='bandstop', output='sos')
        # Apply the filter to the audio signal
        audio_signal = sosfilt(sos, audio_signal)

    return audio_signal

def get_audio_indices(file_name, apply_filter=False, start_freqs=None, end_freqs=None):
    """
    Load an audio file, optionally apply a bandstop filter, and calculate various acoustic indices.

    Parameters:
    - file_name: string, path to the audio file.
    - apply_filter: boolean, indicates if a bandstop filter should be applied.
    - start_freqs: list of floats, start frequencies for the bandstop filters, used if apply_filter is True.
    - end_freqs: list of floats, end frequencies for the bandstop filters, used if apply_filter is True.

    Returns:
    - indices: list, containing the file name and computed acoustic indices or np.nan in case of an error.
    """

        # Load the audio file and compute the spectrogram
    s, fs = torchaudio.load(str(file_name))
    if s.size()[0] != 1:
        s = torch.unsqueeze(s[0, :], 0)

        # If filtering is applied, filter the signal before computing the spectrogram
    if apply_filter and start_freqs is not None and end_freqs is not None:
        s_filtered = butter_bandstop_filter(s.numpy()[0, :], start_freqs, end_freqs, fs)
        s = torch.tensor(s_filtered.copy()[None, :])  # Add a new axis to make it 2D again
        rms = np.sqrt(np.mean(np.square(s.numpy()[0, :])))
        s = s / rms # Normalize the signal

        # Compute the spectrogram
    Sxx, tn, fn, ext = sound.spectrogram(s[0, :], fs, mode='amplitude')
    Sxx_power, _, _, _ = sound.spectrogram(s[0, :], fs)

        # Calculate acoustic indices
    ACIft_ = indices.ACIft(Sxx)
    ADI = features.acoustic_diversity_index(Sxx, fn, fmax=int(fs/2))
    BETA = features.bioacoustics_index(Sxx, fn, flim=(2000, 8000))
    M = features.temporal_median(s[0, :], mode='hilbert')
    NP = features.number_of_peaks(Sxx_power, fn, slopes=6, min_freq_dist=100, display=False)
    Hf, _ = features.frequency_entropy(Sxx_power)
    Ht = features.temporal_entropy(s.numpy()[0, :], mode='hilbert')
    H = Ht * Hf
    AEI = features.acoustic_eveness_index(Sxx, fn, fmax=int(fs/2))
    NDSI, _, _, _ = features.soundscape_index(Sxx_power, fn, flim_bioPh=(2000, 8000), flim_antroPh=(0, 2000))

        # Compile all indices into a list
    acoustic_indices = [ACIft_, ADI, BETA, M, NP, H, AEI, NDSI]

        # Include file name information in the indices list
    file_info = [str(file_name).split('/')[-1], str(file_name).split('/')[-2]]
    return file_info + acoustic_indices


In [7]:
unique_folders = pd.read_sql_query("SELECT DISTINCT folder FROM audios", conn).folder.unique()
reference_folders = ['SM04', 'SM02']
SIGNAL = "chirp"
BANDWIDTH = 1000

for reference_folder in tqdm(reference_folders, total=len(reference_folders)):
    for x, xfolder in enumerate(unique_folders):
        if xfolder != reference_folder:
            # Configuración de la conexión a la base de datos
            max_attempts = 5
            attempt_delay = 2  # Tiempo de espera inicial en segundos

            for attempt in range(max_attempts):
                try:
                    cfr_query = f"SELECT * FROM comparable_frequency_range WHERE folder_x = ? AND folder_y = ? AND signal = ? AND bandwidth = ?"
                    cfr = pd.read_sql_query(cfr_query, conn, params=(reference_folder, xfolder,SIGNAL, BANDWIDTH))
                    
                    # Deserializar las frecuencias
                    cfr['start_freq'] = cfr['start_freq'].apply(pickle.loads)
                    cfr['end_freq'] = cfr['end_freq'].apply(pickle.loads)
                    
                    # Get the indices for each file
                    xaudios = pd.read_sql_query("SELECT path FROM audios WHERE folder = ? AND signal = ?", conn, params=(xfolder,SIGNAL))
                    yaudios = pd.read_sql_query("SELECT path FROM audios WHERE folder = ? AND signal = ?", conn, params=(reference_folder,SIGNAL))

                    for xpath, ypath in zip(xaudios.path, yaudios.path):
                        # Suponemos que la función get_audio_indices está definida en otro lugar del código
                        rxind = get_audio_indices(xpath)
                        pxind = get_audio_indices(xpath, apply_filter=True, start_freqs=cfr.start_freq.iloc[0], end_freqs=cfr.end_freq.iloc[0])
                        ryind = get_audio_indices(ypath)
                        pyind = get_audio_indices(ypath, apply_filter=True, start_freqs=cfr.start_freq.iloc[0], end_freqs=cfr.end_freq.iloc[0])

                        insert_query = "INSERT INTO acoustic_indices (reference_folder, folder_x, rrf_indices, prf_indices, rx_indices, px_indices, signal, bandwidth) VALUES (?, ?, ?, ?, ?, ?, ?, ?)"
                        cursor.execute(insert_query, (reference_folder, xfolder, pickle.dumps(ryind), pickle.dumps(pyind), pickle.dumps(rxind), pickle.dumps(pxind), SIGNAL, BANDWIDTH))
                        conn.commit()
        
                    # Si llegamos aquí sin excepciones, rompemos el bucle de intentos
                    break

                except sqlite3.OperationalError as e:
                    # Si es el último intento, levanta la excepción
                    if attempt == max_attempts - 1:
                        raise
                    print(f"OperationalError encountered on attempt {attempt+1}: {e}")
                    print(f"Waiting for {attempt_delay} seconds before retrying...")
                    time.sleep(attempt_delay)
                    attempt_delay *= 2  # Incrementar el tiempo de espera para el próximo intento

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
y